<a href="https://colab.research.google.com/github/aritrartira/Football-Statistics-Visualisations/blob/main/src/ShotHeatMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PLAYER_NAME = 'Timo Werner'
PLAYER_ID = 838
SEASON = "2020"

# Installing necessary libraries

In [ ]:
!pip install understat
!pip install aiohttp
!pip install mplsoccer

# Importing Necessary Libraries

In [ ]:
from understat import Understat
import json
import asyncio
import aiohttp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import seaborn as sns
import pandas as pd
from mplsoccer.pitch import VerticalPitch

import nest_asyncio
nest_asyncio.apply()

#Getting data

In [ ]:
async def main():
    global shots
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        shots = await understat.get_player_shots(
            65, {"season": SEASON})

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [ ]:
len(shots)

In [ ]:
X = []
Y = []

for shot in shots:
    X.append(float(shot['X']) * 120)
    Y.append((1 - float(shot['Y'])) * 80)

df = pd.DataFrame({'X':X, 'Y':Y})
df

#Plotting

In [ ]:
def create_pitch(color = 'black'):
    fig=plt.figure(figsize=(13,9))
    ax=fig.add_subplot(111)

    plt.axis('off')

    plt.axvspan(0, 130, 0, 90, color = color)

    plt.plot([0,0], [0, 90], color='white')
    plt.plot([130,130], [0, 90], color='white')
    plt.plot([0,130], [0, 0], color='white')
    plt.plot([0,130], [90, 90], color='white')
    plt.plot([65,65], [0, 90], color='white')

    plt.plot([16.5,16.5], [25,65], color = 'white')
    plt.plot([0,16.5], [25,25], color = 'white')
    plt.plot([0,16.5], [65,65], color = 'white')

    plt.plot([130 - 16.5, 130 - 16.5], [25,65], color = 'white')
    plt.plot([130 - 0,130 - 16.5], [25,25], color = 'white')
    plt.plot([130 - 0,130 - 16.5], [65,65], color = 'white')

    plt.plot([5.5, 5.5], [36,54], color='white')
    plt.plot([0, 5.5], [36,36], color='white')
    plt.plot([0, 5.5], [54,54], color='white')

    plt.plot([130 - 5.5, 130 - 5.5], [36,54], color='white')
    plt.plot([130 - 0, 130 - 5.5], [36,36], color='white')
    plt.plot([130 - 0, 130 - 5.5], [54,54], color='white')

    centreCircle = plt.Circle((65,45),9.15,color="white",fill=False)
    centreSpot = plt.Circle((65,45),0.8,color="white")
    ax.add_patch(centreCircle)
    ax.add_patch(centreSpot)

    leftArc = Arc((11,45),height=18.3,width=18.3,angle=0,theta1=310,theta2=50, color='white')
    rightArc = Arc((130 - 11,45),height=18.3,width=18.3,angle=0,theta1=130,theta2=230, color='white')
    ax.add_patch(leftArc)
    ax.add_patch(rightArc)

    leftPenSpot = plt.Circle((11,45),0.8,color="white")
    rightPenSpot = plt.Circle((119,45),0.8,color="white")
    ax.add_patch(leftPenSpot)
    ax.add_patch(rightPenSpot)

    return fig, ax

In [ ]:
pitch = VerticalPitch(pitch_type='statsbomb', stripe=False, figsize=(10,10), pitch_color='grass', line_color='white', half=True, )
fig, ax = pitch.draw()

kde = sns.kdeplot(df['Y'], df['X'], cmap = 'magma', thresh=0.05, fill = True, alpha = 0.4)

plt.scatter(df['Y'], df['X'], alpha = 0.7)

plt.xlim(-5, 90)
plt.ylim(55, 120)

plt.title(f"{PLAYER_NAME} all shots heatmap {SEASON}-{int(SEASON[-2:]) + 1}", fontdict={'fontsize' : '16', 'family':'Ayuthaya'})